In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
! kaggle datasets list -s Rice

In [ ]:
import numpy as np
import pickle
import cv2
from os import listdir
from sklearn.preprocessing import LabelBinarizer
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation, Flatten, Dropout, Dense
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.preprocessing import image
from keras.preprocessing.image import img_to_array
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [ ]:
EPOCHS = 30
INIT_LR = 1e-3
BS = 32
default_image_size = tuple((256, 256))
image_size = 0
directory_root = '../input/rice-diseases-image-dataset/LabelledRice/'
width=256
height=256
depth=3

In [ ]:
def convert_image_to_array(image_dir):
    try:
        image = cv2.imread(image_dir)
        if image is not None :
            image = cv2.resize(image, default_image_size)   
            return img_to_array(image)
        else :
            return np.array([])
    except Exception as e:
        print(f"Error : {e}")
        return None
image_list, label_list = [], []

In [ ]:
try:
    print("[INFO] Loading images ...")
    root_dir = listdir(directory_root)
    for directory in root_dir :
        # remove .DS_Store from list
        if directory == ".DS_Store" :
            root_dir.remove(directory)

    for plant_folder in root_dir :
        plant_disease_folder_list = listdir(f"{directory_root}/{plant_folder}")
        
        for disease_folder in plant_disease_folder_list :
            # remove .DS_Store from list
            if disease_folder == ".DS_Store" :
                plant_disease_folder_list.remove(disease_folder)

        for plant_disease_folder in plant_disease_folder_list:
            print(f"[INFO] Processing {plant_disease_folder} ...")
            plant_disease_image_list = listdir(f"{directory_root}/{plant_folder}/{plant_disease_folder}/")
                
            for single_plant_disease_image in plant_disease_image_list :
                if single_plant_disease_image == ".DS_Store" :
                    plant_disease_image_list.remove(single_plant_disease_image)

            for image in plant_disease_image_list:
                image_directory = f"{directory_root}/{plant_folder}/{plant_disease_folder}/{image}"
                if image_directory.endswith(".jpg") == True or image_directory.endswith(".JPG") == True:
                    image_list.append(convert_image_to_array(image_directory))
                    label_list.append(plant_disease_folder)
    print("[INFO] Image loading completed")  
except Exception as e:
    print(f"Error : {e}")

In [ ]:
image_size = len(image_list)
print(image_size)

In [ ]:
label_binarizer = LabelBinarizer()
image_labels = label_binarizer.fit_transform(label_list)
pickle.dump(label_binarizer,open('label_transform.pkl', 'wb'))
n_classes = len(label_binarizer.classes_)

In [ ]:
print(label_binarizer.classes_)
np_image_list = np.array(image_list, dtype=np.float16) / 225.0
print("[INFO] Spliting data to train, test")
x_train, x_test, y_train, y_test = train_test_split(np_image_list, image_labels, test_size=0.30, random_state = 42) 
aug = ImageDataGenerator(
    rotation_range=30, width_shift_range=0.15,
    height_shift_range=0.15, shear_range=0.15, 
    zoom_range=0.2,horizontal_flip=True, 
    fill_mode="nearest")
from torchvision import datasets, transforms,models

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.applications.vgg16 import VGG16

In [ ]:
# Download the inception v3 weights
# !wget --no-check-certificate \
#     https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
#     -O /tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
# res = os.system(command)

# Import the inception model  
from tensorflow.keras.applications.vgg16 import VGG16

# Create an instance of the inception model from the local pre-trained weights
# local_weights_file = '/tmp/NASNet-large-no-top.h5'

pre_trained_model = VGG16(input_shape=(256,256,3), include_top=False,weights='imagenet')

# pre_trained_model.load_weights(local_weights_file)

# Make all the layers in the pre-trained model non-trainable
for layer in pre_trained_model.layers:
  # Your Code Here
  layer.trainable=False
  
# Print the model summary
pre_trained_model.summary()


In [ ]:
# Define a Callback class that stops training once accuracy reaches 99.9%
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('acc')>0.90 and logs.get('val_acc')>0.89):
      print("\nReached 99.9% accuracy and 99% validation accuracy so cancelling training!")
      self.model.stop_training = True


In [ ]:
# from tensorflow.keras.optimizers import RMSprop

# # Flatten the output layer to 1 dimension
# x = layers.Flatten()(pre_trained_model.output)
# # Add a fully connected layer with 1,024 hidden units and ReLU activation
# x = layers.Dense(1024, activation='relu')(x)
# # Add a dropout rate of 0.2
# x = layers.Dropout(0.2)(x)                  
# # Add a final sigmoid layer for classification
# x = layers.Dense(n_classes, activation='softmax')(x)   

In [ ]:
# model = Model(pre_trained_model.input, x) 
# opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
# model.compile(optimizer = 'adam', 
#               loss = 'binary_crossentropy', 
#               metrics = ['acc'])

# # model.summary()

In [ ]:
# from keras.callbacks import ModelCheckpoint

# checkpoint = ModelCheckpoint('best_model.h5', verbose=1, monitor='acc', save_best_only=True, mode='auto') 

In [ ]:

# callbacks = myCallback()
# history = model.fit(
#     aug.flow(x_train, y_train, batch_size=BS),
#     validation_data=(x_test, y_test),
#     steps_per_epoch=len(x_train) // BS,
#     epochs=50, verbose=1,
#     callbacks=[callbacks]
#     )

In [ ]:
# import matplotlib.pyplot as plt
# acc = history.history['acc']
# val_acc = history.history['val_acc']
# loss = history.history['loss']
# val_loss = history.history['val_loss']

# epochs = range(len(acc))

# plt.plot(epochs, acc, 'r', label='Training accuracy')
# plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
# plt.title('Training and validation accuracy')
# plt.legend(loc=0)
# plt.figure()


# plt.show()

In [ ]:
# import tensorflow as tf

# # saving your deep learning model
# model.save('plant_ai_model.h5')
# # Convert the model.
# converter = tf.lite.TFLiteConverter.from_keras_model(model)
# tflite_model = converter.convert()
# open("plant_ai.tflite", "wb").write(tflite_model)

In [ ]:
# MobileNet
from keras import layers , Input , Model

def stem( inputs , alpha ):
    filters = int( 32 * alpha )
    # Normal Convolutional block
    x = layers.ZeroPadding2D( padding =(( 0 , 1 ), ( 0 , 1 )))( inputs )
    x = layers.Conv2D( filters , ( 3 , 3 ), strides =( 2 , 2 ), padding = 'valid' )( x )
    x = layers.BatchNormalization()( x )
    x = layers.ReLU()( x )
    # Depthwise Separable Convolution Block
    x = depthwise_block( x , 64 , alpha , ( 1 , 1 ))
    return x

def classifier( x , alpha , dropout , nb_classes ):
    """ Create the classifier group
    inputs : input tensor
    alpha : width multiplier
    dropout : dropout percentage
    nb_classes: number of output classes
    """
    # Flatten the feature maps into 1D feature maps (?, N)
    x = layers.GlobalAveragePooling2D()( x )
    # Reshape the feature maps to (?, 1, 1, 1024)
    shape = ( 1 , 1 , int( 1024 * alpha ))
    x = layers.Reshape( shape )( x )
    # Perform dropout for preventing overfitting
    x = layers.Dropout( dropout )( x )
    # Use convolution for classifying (emulates a fully connected layer)
    x = layers.Conv2D( nb_classes , ( 1 , 1 ), padding = 'same' )( x )
    x = layers.Activation( 'softmax' )( x )
    # Reshape the resulting output to 1D vector of number of classes
    x = layers.Reshape(( nb_classes , ))( x )
    return x


def depthwise_block( x , nb_filters , alpha , strides ):
    """ Create a Depthwise Separable Convolution block
    inputs : input tensor
    nb_filters: number of filters
    alpha : width multiplier
    strides : strides
    """
    # Apply the width filter to the number of feature maps
    filters = int( nb_filters * alpha )
    if strides == ( 2 , 2 ):
        x = layers.ZeroPadding2D( padding =(( 0 , 1 ), ( 0 , 1 )))( x )
        padding = 'valid'
    else :
        padding = 'same'
    # Depthwise Convolution
    x = layers.DepthwiseConv2D (( 3 , 3 ), strides , padding = padding )( x )
    x = layers.BatchNormalization ()( x )
    x = layers.ReLU ()( x )
    # Pointwise Convolution
    x = layers.Conv2D ( filters , ( 1 , 1 ), strides =( 1 , 1 ), padding = 'same' )( x )
    x = layers.BatchNormalization ()( x )
    x = layers.ReLU ()( x )
    return x





In [ ]:
import keras
keras.backend.clear_session() 
alpha = 1 # width multiplier
dropout = 0.5 # dropout percentage
nb_classes = n_classes # number of classes
inputs = Input ( shape =( 256 , 256 , 3 ))
# Create the stem group
x = stem( inputs , alpha )
# First Depthwise Separable Convolution Group
# Strided convolution - feature map size reduction
x = depthwise_block( x , 128 , alpha , strides =( 2 , 2 ))
x = depthwise_block( x , 128 , alpha , strides =( 1 , 1 ))
# Second Depthwise Separable Convolution Group
# Strided convolution - feature map size reduction
x = depthwise_block( x , 256 , alpha , strides =( 2 , 2 ))
x = depthwise_block( x , 256 , alpha , strides =( 1 , 1 ))
# Third Depthwise Separable Convolution Group
# Strided convolution - feature map size reduction
x = depthwise_block( x , 512 , alpha , strides =( 2 , 2 ))
for _ in range( 5 ):
    x = depthwise_block( x , 512 , alpha , strides =( 1 , 1 ))
    # Fourth Depthwise Separable Convolution Group
    # Strided convolution - feature map size reduction

In [ ]:
x = depthwise_block ( x , 1024 , alpha , strides =( 2 , 2 ))
x = depthwise_block ( x , 1024 , alpha , strides =( 1 , 1 ))
x = classifier ( x , alpha , dropout , nb_classes )
print( x )
model = Model ( inputs , x )

model.compile(optimizer = 'adam', 
              loss = 'binary_crossentropy', 
              metrics = ['acc'])

model.summary ()

In [ ]:


callbacks = myCallback()
history = model.fit(
    aug.flow(x_train, y_train, batch_size=BS),
    validation_data=(x_test, y_test),
    steps_per_epoch=len(x_train) // BS,
    epochs=300, verbose=1,
    callbacks=[callbacks]
    )

In [ ]:
model.save('plant_ai_model.pb')

In [ ]:
import tensorflow as tf

# saving your deep learning model
model.save('plant_ai_model.h5')
# # Convert the model.
# converter = tf.lite.TFLiteConverter.from_keras_model(model)
# tflite_model = converter.convert()
# open("plant_ai.tflite", "wb").write(tflite_model)

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()


plt.show()

In [ ]:
# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
open("plant_ai.tflite", "wb").write(tflite_model)

In [ ]:
# converter = tf.lite.TFLiteConverter.from_saved_model('../output/kaggle/working/')
# tflite_model = converter.convert()
# open("plant_ai.tflite", "wb").write(tflite_model)